# Coding Applications in Medicine: Data Science - Regression Analysis

Regression analysis are used to depict the relationship between multiple variables. There are two basic types of regression models, linear regression and logistic regression. 

Linear regression is commonly used for value prediction (numerical), while logistic regression is commonly used for data classification (categorical).

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import RocCurveDisplay

In [ ]:
# Load the insurance data.
insuranceDF = pd.read_csv("data/insurance.csv")
# Create a new column with smoker column as a numerical value.
insuranceDF["smoker numerical"] = (insuranceDF["smoker"] == "yes").astype(int)
insuranceDF

## Linear Regression

Linear regression is one way to model the relationship between a dependent variable and one or multiple independent variable. When constructing a linear model, we are attempting to fit the data into the formula $\hat{Y}=X\theta$, where $\hat{Y}$ is the prediction vector, $X$ is the design matrix, and $\theta$ is the parameter vector. There are different approaches to create such model (or equation) mathematically. 

The goal is to use the existing data $X$ and $Y$ data to generate the optimal $\theta$ used for later prediction.

Note: $\hat{Y}$ is a matrix of size $n\times1$. $X$ is a matrix of size $n\times(p+1)$. $\theta$ is a matrix of size $(p+1)\times n$ 

You can think of $\hat{Y}=X\theta$ as the classic $y = mx + b$ but now expanded to cover multiple $x$ (independent variables) and to make predictions $y$ (dependent variable) for multiple people at once.

Suppose you wish to explore the relationship between age and insurance charges.

In [ ]:
sns.scatterplot(data=insuranceDF, x="age", y="charges")
plt.show();

What would we expect based on the plot?

In [ ]:
# Split our dataset to training and testing datasets.
linData1_tr, linData1_te = train_test_split(insuranceDF, test_size=0.1, random_state=42)
linData1_tr.reset_index(inplace=True, drop=True)
linData1_te.reset_index(inplace=True, drop=True)

# Create the X and Y matrix for model training.
linTraining1X = linData1_tr[["age"]].to_numpy()
linTraining1Y = linData1_tr["charges"].to_numpy()

# Creating the linear regression model.
linModel1 = LinearRegression(fit_intercept=True)
linModel1.fit(linTraining1X, linTraining1Y)

# Resulting linear regression model parameters.
linModel1.intercept_, linModel1.coef_

In [ ]:
# Plot of the data with the linear regression line.
sns.scatterplot(data=insuranceDF, x="age", y="charges")
# Plot of the linear regression line.
plt.plot(np.linspace(18, 64, 1000), 
         linModel1.predict(np.reshape(np.linspace(18, 64, 1000), (-1, 1))), color="red")
plt.show();

In [ ]:
# Use the model to make predictions (Age).
linModel1.predict([[25]])

In [ ]:
# The score tells you the accuracy of the regression model.
# You should use the test dataset for the score calculation.
linModel1.score(linData1_te[["age"]].to_numpy(), linData1_te["charges"].to_numpy())

This time, we want to explore the relationship between all the numerical variables and insurance charges.

In [ ]:
# Scatterplot capturing four features/dimensions.
sns.scatterplot(data=insuranceDF, x="age", y="charges", hue="smoker", size="bmi");
plt.legend(loc="center left", bbox_to_anchor=(1, 0.5))
plt.show();

What would we expect based on the plot?

In [ ]:
# Split our dataset to training and testing datasets.
linData2_tr, linData2_te = train_test_split(insuranceDF, test_size=0.1, random_state=24)
linData2_tr.reset_index(inplace=True, drop=True)
linData2_te.reset_index(inplace=True, drop=True)

# Create the X and Y matrix for model training.
linTraining2X = linData2_tr.drop(["sex", "charges", "smoker", "region"], axis=1).to_numpy()
linTraining2Y = linData2_tr["charges"].to_numpy()

# Creating the linear regression model.
linModel2 = LinearRegression(fit_intercept=True)
linModel2.fit(linTraining2X, linTraining2Y)

# Resulting linear regression model parameters.
linModel2.intercept_, linModel2.coef_

In [ ]:
# Use the model to make predictions (Age, BMI, # Children, Smoker - 0 or 1).
linModel2.predict([[30, 25, 0, 0]])

In [ ]:
# The score tells you the accuracy of the regression model.
# You should use the test dataset for the score calculation.
linModel2.score(linData2_te.drop(["sex", "charges", "smoker", "region"], axis=1).to_numpy(),
                linData2_te["charges"].to_numpy())

For more information, check the following:
- User Guide: https://scikit-learn.org/stable/modules/linear_model.html
- API Reference: https://scikit-learn.org/stable/modules/classes.html#module-sklearn.linear_model
- Examples: https://scikit-learn.org/stable/auto_examples/index.html#generalized-linear-models

## Logistic Regression

Logistic regression is a model used to estimate the probability that an event occurred given previous data. When the probability exceeds the threshold value, we considered the event as having occurred. Essentially, logistic regression is used for binary (True/False) classification. This is done by the formula $\hat{P}_\theta (Y = 1|x) = \sigma(x^T\theta)$, where $\theta$ is the linear parameter, $\sigma$ is the sigmoid or logistic function, $x$ is the input feature matrix, and $y$ is the response vector.

Suppose you wish to determine if you can predict whether a person is a smoker based on their insurance charges.

In [ ]:
sns.histplot(data=insuranceDF, x="charges", hue="smoker", binwidth=1000, kde=True);
plt.show();

What would we expect based on the plot?

In [ ]:
# A stripplot is a categorical scatterplot that uses jittering to reduce over plotting.
sns.stripplot(data=insuranceDF, x="charges", y="smoker numerical", jitter=0.1, orient='h').invert_yaxis()
plt.show();

Here is a different plot depicting the same data.

In [ ]:
# Split our dataset to training and testing datasets.
logData1_tr, logData1_te = train_test_split(insuranceDF, test_size=0.1, random_state=21)
logData1_tr.reset_index(inplace=True, drop=True)
logData1_te.reset_index(inplace=True, drop=True)

# Create the X and Y matrix for model training.
logTraining1X = logData1_tr[["charges"]].to_numpy()
logTraining1Y = logData1_tr["smoker numerical"].to_numpy()

# Creating the logistic regression model.
logModel1 = LogisticRegression(fit_intercept=True)
logModel1.fit(logTraining1X, logTraining1Y)

# Resulting logistic regression model parameters.
logModel1.intercept_, logModel1.coef_

In [ ]:
# Sigmoid or logistic function.
def sigmoid(x):
    return 1/(1 + np.exp(-x))

log_theta_0 = logModel1.intercept_[0]
log_theta_1 = logModel1.coef_[0]

# Does the prediction match the actual data.
log_smoke_pred_match = logModel1.predict(logData1_tr[["charges"]].to_numpy())==logData1_tr["smoker numerical"]

# Plot of the training data.
sns.stripplot(x=logData1_tr["charges"], y=logData1_tr["smoker numerical"],
              jitter=0.1, orient='h', hue=log_smoke_pred_match).invert_yaxis()

# Plot of the logistic regression line.
sns.lineplot(x=np.linspace(0, 70000, 1000), y=sigmoid(log_theta_0 + log_theta_1 * np.linspace(0, 70000, 1000)));

In [ ]:
# Use the model to make predictions (Charges).
logModel1.predict([[30000]])

In [ ]:
# The score tells you the accuracy of the regression model.
# You should use the test dataset for the score calculation.
logModel1.score(logData1_te[["charges"]].to_numpy(), logData1_te["smoker numerical"].to_numpy())

This time, we will try to predict if someone is a smoker using all the numerical variables.

In [ ]:
# Scatterplot with introduction of jitter.
sns.scatterplot(data=insuranceDF.assign(smoker_jitter=insuranceDF["smoker numerical"] 
                                        + np.random.uniform(-0.2, 0.2, len(insuranceDF))),
                x="charges", y="smoker_jitter", hue="age", size="bmi", style="children",
                sizes=(20, 200))
plt.legend(loc="center left", bbox_to_anchor=(1, 0.5))
plt.show();

In [ ]:
# Split our dataset to training and testing datasets.
logData2_tr, logData2_te = train_test_split(insuranceDF, test_size=0.1, random_state=12)
logData2_tr.reset_index(inplace=True, drop=True)
logData2_te.reset_index(inplace=True, drop=True)

# Create the X and Y matrix for model training.
logTraining2X = logData2_tr.drop(["sex", "smoker", "region", "smoker numerical"], axis=1).to_numpy()
logTraining2Y = logData2_tr["smoker numerical"].to_numpy()

# Creating the logistic regression model.
logModel2 = LogisticRegression(fit_intercept=True)
logModel2.fit(logTraining2X, logTraining2Y)

# Resulting logistic regression model parameters.
logModel2.intercept_, logModel2.coef_

In [ ]:
# Use the model to make predictions (Age, BMI, # Children, Insurance Charges).
logModel2.predict([[30, 25, 0, 3000]])

In [ ]:
# Create the X and Y matrix for model testing.
logTest2X = logData2_te.drop(["sex", "smoker", "region", "smoker numerical"], axis=1).to_numpy()
logTest2Y = logData2_te["smoker numerical"].to_numpy()

# The score tells you the accuracy of the regression model.
# You should use the test dataset for the score calculation.
logModel2.score(logTest2X, logTest2Y)

For more information, check the following:
- User Guide: https://scikit-learn.org/stable/modules/linear_model.html
- API Reference: https://scikit-learn.org/stable/modules/classes.html#module-sklearn.linear_model
- Examples: https://scikit-learn.org/stable/auto_examples/index.html#generalized-linear-models

## ROC Curves 

The ROC curve is used to show how well a diagnostic test can distinguish between two different groups. This is useful in evaluating the quality of our classifier model (in this case, the logistic regression model).

In [ ]:
# Get the probability estimates of the classification, used in ROC curve.
smoker_pred = logModel2.predict_proba(logTest2X)[:, 1]

# Display the ROC curve for the logistic regression.
RocCurveDisplay.from_predictions(logTest2Y, smoker_pred, name="Log Regression")
# Produces the same result as above.
### RocCurveDisplay.from_estimator(logModel2, logTest2X, logTest2Y, name="Log Regression")

# No predictive value curve.
plt.plot([0, 1], [0, 1], "k--")
plt.title("ROC Curve")
plt.show();

References:
- https://scikit-learn.org/stable/modules/generated/sklearn.metrics.RocCurveDisplay.html
- https://scikit-learn.org/stable/modules/generated/sklearn.metrics.roc_curve.html
- https://scikit-learn.org/stable/modules/generated/sklearn.metrics.roc_auc_score.html

**Source:**


Module adapted from Kaggle: https://www.kaggle.com/code/mariapushkareva/medical-insurance-cost-with-linear-regression/notebook

Dataset source: https://github.com/stedy/Machine-Learning-with-R-datasets